<a href="https://colab.research.google.com/github/LabCInf/Data-Analysis-Basic/blob/main/Valores_at%C3%ADpicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

<h1> Recurso educativo: limpieza de datos</h1>

<h2>Material preparado por: Juan Fernando Pérez</h2>

Recuerden que lo más importante siempre será **¡prácticar, prácticar y prácticar!**


# Librerías

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
from pandas.core.frame import DataFrame
#Cargar los datos
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Materia Gestión y análisis de datos/winequality-white.csv', sep=";",encoding="utf-8-sig")
df

FileNotFoundError: ignored

In [ ]:
df.mean()

fixed acidity             6.854788
volatile acidity          0.278241
citric acid               0.334192
residual sugar            6.391415
chlorides                 0.045772
free sulfur dioxide      35.308085
total sulfur dioxide    138.360657
density                   0.994027
pH                        3.188267
sulphates                 0.489847
alcohol                  10.514267
quality                   5.877909
dtype: float64

# Graficar los valores atipicos

**Histograma**

Usando un histograma, podemos ver cómo se distribuyen los datos. Tener datos que siguen una distribución normal es necesario para algunas de las técnicas estadísticas utilizadas para detectar valores atípicos.

In [ ]:
#crear un histograma
fig = px.histogram(df, x='total sulfur dioxide') #reemplazar el nombre de la columna que queremos analizar en las comillas simples
fig.show()

**Diagrama de caja que muestra valores atípicos univariados**

Un diagrama de caja nos permite identificar los valores atípicos univariados o valores atípicos para una variable. Los diagramas de caja son útiles porque muestran valores mínimos y máximos, la mediana y el rango intercuartílico de los datos. En el gráfico, los valores atípicos se muestran como puntos, lo que los hace fáciles de ver.

In [ ]:
#crear diagrama de caja
fig = px.box(df, y="total sulfur dioxide") #reemplazar el nombre de la columna que queremos analizar en las comillas simples
fig.show()

**Gráfico de dispersión que muestra valores atípicos multivariados**

Usando un gráfico de dispersión, es posible revisar los valores atípicos multivariados, o los valores atípicos que existen en dos o más variables

In [ ]:
fig = px.scatter(x=df['free sulfur dioxide'], y=df['total sulfur dioxide']) #reemplazar el nombre de la columna que queremos analizar en las comillas simples
fig.show()

# **Encontrar valores atípicos usando métodos estadísticos**

Usando la conveniente función pandas .quantile() , podemos crear una función de Python simple que toma nuestra columna del marco de datos y genera los valores atípicos:

In [ ]:
def find_outliers_IQR(df):
    q1=df.quantile(0.25)
    q3=df.quantile(0.75)
    IQR=q3-q1
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
    return outliers

In [ ]:
outliers = find_outliers_IQR(df["total sulfur dioxide"]) #reemplazamos el nombre de la columna de la cual queremos ver los valores atipicos en estas comillas dobles
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))
outliers

number of outliers: 19
max outlier value: 440.0
min outlier value: 9.0


227     272.0
325     313.0
387     260.0
740      19.0
1417    366.5
1931    307.5
1940    256.0
1942    256.0
2127    344.0
2378    282.0
2654    303.0
3050    272.0
3094     18.0
3095     18.0
3152    294.0
3710      9.0
3901     10.0
4514    259.0
4745    440.0
Name: total sulfur dioxide, dtype: float64

In [ ]:
upper_limit = df['total sulfur dioxide'].mean() + 3*df['total sulfur dioxide'].std() #reemplazar el nombre de la columna que queremos analizar en las comillas simples
print(upper_limit)
lower_limit = df['total sulfur dioxide'].mean() - 3*df['total sulfur dioxide'].std()
print(lower_limit)

265.85485107361706
10.866463748759415


# **Reemplazar valores atípicos usando imputación**

Utilice cualquier técnica de imputación que se esté utilizando en los valores faltantes. Por ejemplo, si la columna "total sulfur dioxide" tuviera valores faltantes, podríamos considerar apropiado completar los valores faltantes usando la media. Dado que así es como tratamos los valores faltantes, haríamos lo mismo con los valores atípicos.

Use una función para encontrar los valores atípicos usando IQR y reemplácelos con el valor medio. Nómbrelo "impute_outliers_IQR". En la función, podemos obtener un límite superior y un límite inferior usando el . máx() y . min() funciones respectivamente. Luego podemos usar numpy .where() para reemplazar los valores.

Podemos pasar 'total sulfur dioxide' a través de la función "impute_outliers_IQR" para transformar los valores atípicos en el valor medio. Podemos usar .describe() para verificar que la función funcione.

In [ ]:
def impute_outliers_IQR(df):
    q1=df.quantile(0.25)
    q3=df.quantile(0.75)
    IQR=q3-q1
    upper = df[~(df>(q3+1.5*IQR))].max()
    lower = df[~(df<(q1-1.5*IQR))].min()
    df = np.where(df > upper, 
        df.mean(),
        np.where(
            df < lower,
            df.mean(),
            df
            )
        )
    return df

In [ ]:
df['total sulfur dioxide'] = impute_outliers_IQR(df['total sulfur dioxide']) #reemplazar el nombre de la columna que queremos analizar en las comillas simples
df.describe()['total sulfur dioxide']

count    4898.000000
mean      138.019672
std        41.313592
min        21.000000
25%       108.000000
50%       134.000000
75%       167.000000
max       255.000000
Name: total sulfur dioxide, dtype: float64

In [ ]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [ ]:
df["total sulfur dioxide"][3710] #Corroboramos los valores atipicos

138.36065741118824

# Guardar los nuevo datos

In [ ]:
#Realizamos este procedimiento para guardar los datos nuevos, recuerden cambiar el nombre del archivo original para no sobreescribir
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Materia Gestión y análisis de datos/winequality-white_2.csv', sep=";",encoding="utf-8-sig")

In [ ]:
#Cargar los datos
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Materia Gestión y análisis de datos/winequality-white_2.csv', sep=";",encoding="utf-8-sig")
df1

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [ ]:
#Aplicamos nuevamente la función para identificar nuevante si los datos nuevos tienen valores atípicos

outliers = find_outliers_IQR(df1["total sulfur dioxide"])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))
outliers

number of outliers: 0
max outlier value: nan
min outlier value: nan


Series([], Name: total sulfur dioxide, dtype: float64)